# Чтение данных

In [1]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [2]:
df_list = []

for i in range(12):
    df_list.append(
        pd.read_csv(f"columns/out_{i}.csv", sep="⇧", engine="python", quotechar='"', on_bad_lines="warn")
    )
df = pd.concat(df_list, axis=0)
df

Skipping line 48925: '⇧' expected after '"'
Skipping line 48926: '⇧' expected after '"'
Skipping line 48927: '⇧' expected after '"'
Skipping line 343862: field larger than field limit (131072)


,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,Чарт (2019)
1,dataset/Dataset/data/7779522/table_0_meta.json,1,Высшаяпозиция
2,dataset/Dataset/data/2863377/table_11_meta.json,0,Unnamed: 0
3,dataset/Dataset/data/2863377/table_11_meta.json,1,Команда
4,dataset/Dataset/data/2863377/table_11_meta.json,2,И
...,...,...,...
409524,dataset/Dataset/data/35185/table_5_meta.json,2,Дата получения Любекского права
409525,dataset/Dataset/data/35185/table_5_meta.json,3,Дата получения Кульмского права
409526,dataset/Dataset/data/35185/table_5_meta.json,4,Дата получения Магдебургского права
409527,dataset/Dataset/data/35185/table_5_meta.json,5,Дата вступления в союз


# Research
---
Проблемные столбцы (скорее всего не будем использовать):
- Столбцы по типу: "ΔHкип, кДж·моль−1"
- Столбцы использующиеся для группировки каких-либо значений по времени (месяцу, году и т.п.)
- Столбцы без имени
- Столбцы, название которых не имеет смысла (числа и т.п.)
- Аббревиатуры и сокращения

### Столбцы без названия = 164500
их можно отправить в тестовую выборку, чтобы не терять информацию. (нужно еще посмотреть, что внутри них обычно лежит)

### Без имени
можно в тест

## .mv-parser... = 15661
обрезать только осмысленный текст

---

In [3]:
df["column_name"].value_counts()

column_name
Год                   220620
Место                  87174
Роль                   78968
Команда                67699
Название               64943
                       ...  
Пб за игру                 1
Имя божества               1
Культура / Религия         1
Ə ə                        1
Норте (Norte)              1
Name: count, Length: 125687, dtype: int64

## Столбцы без названия

In [4]:
unnamed_columns = df["column_name"].apply(lambda x: str(x).startswith("Unnamed"))
df[unnamed_columns]

,table_id,column_id,column_name
2,dataset/Dataset/data/2863377/table_11_meta.json,0,Unnamed: 0
13,dataset/Dataset/data/4727026/table_0_meta.json,1,Unnamed: 1
17,dataset/Dataset/data/5212502/table_3_meta.json,0,Unnamed: 0
79,dataset/Dataset/data/473332/table_0_meta.json,1,Unnamed: 1
89,dataset/Dataset/data/3702093/table_0_meta.json,3,Unnamed: 3
...,...,...,...
409331,dataset/Dataset/data/171226/table_9_meta.json,4,Unnamed: 4
409366,dataset/Dataset/data/367669/table_11_meta.json,0,Unnamed: 0
409458,dataset/Dataset/data/730442/table_14_meta.json,12,Unnamed: 12
409459,dataset/Dataset/data/730442/table_14_meta.json,13,Unnamed: 13


## Столбцы содержащие в названии: ".mw-parser..."

In [5]:
mv_parser = df["column_name"].apply(lambda x: str(x).strip().startswith(".mw-parser"))
df[mv_parser]

,table_id,column_id,column_name
151,dataset/Dataset/data/8071239/table_3_meta.json,0,.mw-parser-output .ts-comment-commentedText{bo...
983,dataset/Dataset/data/6799868/table_6_meta.json,0,.mw-parser-output .ts-comment-commentedText{bo...
1083,dataset/Dataset/data/6798417/table_0_meta.json,0,.mw-parser-output .ts-comment-commentedText{bo...
1284,dataset/Dataset/data/930522/table_0_meta.json,1,.mw-parser-output .ts-comment-commentedText{bo...
1593,dataset/Dataset/data/6426908/table_0_meta.json,6,.mw-parser-output .ts-comment-commentedText{bo...
...,...,...,...
406936,dataset/Dataset/data/7805760/table_2_meta.json,0,.mw-parser-output .ts-comment-commentedText{bo...
407609,dataset/Dataset/data/6401706/table_0_meta.json,6,.mw-parser-output .ts-comment-commentedText{bo...
408191,dataset/Dataset/data/6822002/table_43_meta.json,0,.mw-parser-output .ts-comment-commentedText{bo...
408375,dataset/Dataset/data/3100187/table_0_meta.json,0,.mw-parser-output .ts-comment-commentedText{bo...


### В назнваниях столбца содержится человекопонятное название, нужно отчистить мусорную информацию

In [6]:
mv_parser_humnareadable_names = df[mv_parser]["column_name"].apply(lambda x: str(x).split("}")[-1])
mv_parser_humnareadable_names.unique()

array(['М', 'Лига', 'Депонирование[1]', 'И', ' В', 'В', '№', 'Ист.',
       'Жертвы', '15+15 км', 'И.', 'Год', 'Кат', '±', 'Поз.',
       'Синяя сторона', 'ПЧ', 'Население[65]', 'Категория', 'Σ',
       'Оценка D', 'Bib', 'Этап', 'Игр', 'Л/Круги', 'ОН', 'П', '03/04',
       'В-П', '02/03', 'СС', '06/07', 'Пал.', 'С', '11/12', 'БК', 'Д / В',
       'Прим.', ' И', 'ХП', 'Поз', 'Пр.', 'СУ', 'Л.круги', '08/09', 'П.',
       'З', '#', 'КТ', 'ПП', 'Pos.', 'Уров.', 'Кандзи', 'ТТХ',
       '7,5+7,5 км', 'Плотность(чел./км²)', 'Поб', 'PPG',
       'Численностьверующихмлн. человек', 'Портрет', 'Финал', 'КН',
       'Доступ-ность', 'Группа', 'Ref', 'WR', 'О', 'Кубок',
       'Оригинальное название', 'Н', 'Период', 'Примечание',
       'Дата рождения', 'СЭ', 'КЧ', 'Очки', 'Также вышел', 'Т',
       'Название в заявкена номинацию', 'Звание', 'Место', 'Див.', 'No.',
       'Голосовало', 'SPG', 'Сейчас', 'Гр.', 'А-2010', 'Оч', 'В–П',
       ' П/У', 'APG',
       'Доля учеников по обучению на русском 

## Аббревиатуры / сокращения / меры (кг, кдж.)

In [7]:
# TODO: what should i do? -_-

## Заголовки, состоящие из одной буквы

In [8]:
no_word_in_header = df["column_name"].str.len() < 2
df[no_word_in_header]["column_name"].unique()

array(['И', 'В', 'Н', 'П', '+', '−', '=', '#', '±', 'О', '№', 'Ш', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', 'M', 'М', 'Г', 'С', 'B',
       'F', 'm', '%', 'W', 'L', '-', 'N', 'V', 'E', 'Б', 'Δ', 'Д', 'н',
       'К', 'З', '0', 'Р', 'Я', '«', '»', '學', 'Т', 'А', 'T', 'I', 'Ф',
       'Л', 'A', 'C', 'D', 'ä', 'i', 'a', 'e', 'ə', 'G', 'అ', 'ఆ', 'ఇ',
       'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ౠ', 'ఌ', 'ౡ', 'ఎ', 'ఏ', 'ఐ', 'ఒ', 'ఓ', 'ఔ',
       'Á', 'Ⱥ', 'Ć', 'Ȼ', 'H', 'n', 'b', 'c', 'd', 'f', 'g', 'h', 'k',
       'l', 'P', 'K', 'y', '…', '×', 'χ', 's', '_', 'â', 'p', '*', 'R',
       'X', '•', 'u', 'o', 'Э', 'Σ', 'Ж', 'ε', 'Z', 'J', 'O', 'Q', 'S',
       'U', 'Y', 'q', 'Ч', 'x', 'z', 'α', 'β', 'δ', 'ʾ', 'w', 'ʿ', 'ṭ',
       'ṣ', 'ḍ', 'Х', 'У', 'j', 't', 'ʼ', 'κ', 'क', 'ख', 'ग', 'घ', 'ङ',
       'с', '∧', 'Е', 'Ь', '—', 'ښ', 'ږ', 'څ', 'ځ', 'ژ', 'ā', 'ꤢ', 'ꤣ',
       'ꤤ', 'ꤥ', '<', '>', 'я', 'ф', 'м', 'а', 'и', 'о', 'д', 'ʤ', '𑄶',
       '𑄷', '𑄸', '𑄹', '𑄺', '𑄻', '𑄼', '𑄽', '𑄾', '𑄿', '’', '↕', 'अ

# Очистка данных

---
1. Очистка мусора из столбцов, содержащих ".mv-parser.."
2. Выделение столбцов без заголовка
3. Замена специальных символов на слова (№, $, %, и т.п.)
4. Очистка символов не из русского алфавита
5. Удаление лишних пробелов с начала и конца заголовка
---

In [9]:
df_cleansed = df.copy()

## 1. Очистка мусора из столбцов

In [10]:
def clean_mv_parser(col_name: str) -> str:
    if col_name.strip().startswith(".mw-parser"):
        return col_name.split("}")[-1]
    return col_name

df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lambda x: clean_mv_parser(str(x)))

100%|████████████████████████████| 4966161/4966161 [00:02<00:00, 2454835.55it/s]


In [11]:
df_cleansed[mv_parser]

,table_id,column_id,column_name
151,dataset/Dataset/data/8071239/table_3_meta.json,0,М
983,dataset/Dataset/data/6799868/table_6_meta.json,0,М
1083,dataset/Dataset/data/6798417/table_0_meta.json,0,М
1284,dataset/Dataset/data/930522/table_0_meta.json,1,Лига
1593,dataset/Dataset/data/6426908/table_0_meta.json,6,Депонирование[1]
...,...,...,...
406936,dataset/Dataset/data/7805760/table_2_meta.json,0,М
407609,dataset/Dataset/data/6401706/table_0_meta.json,6,Доступ-ность
408191,dataset/Dataset/data/6822002/table_43_meta.json,0,М
408375,dataset/Dataset/data/3100187/table_0_meta.json,0,М


## 2. Выделение столбцов без заголовка

In [12]:
unnamed_columns = df_cleansed["column_name"].str.startswith("Unnamed")
df_cleansed = df_cleansed[~unnamed_columns]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,Чарт (2019)
1,dataset/Dataset/data/7779522/table_0_meta.json,1,Высшаяпозиция
3,dataset/Dataset/data/2863377/table_11_meta.json,1,Команда
4,dataset/Dataset/data/2863377/table_11_meta.json,2,И
5,dataset/Dataset/data/2863377/table_11_meta.json,3,В
...,...,...,...
409524,dataset/Dataset/data/35185/table_5_meta.json,2,Дата получения Любекского права
409525,dataset/Dataset/data/35185/table_5_meta.json,3,Дата получения Кульмского права
409526,dataset/Dataset/data/35185/table_5_meta.json,4,Дата получения Магдебургского права
409527,dataset/Dataset/data/35185/table_5_meta.json,5,Дата вступления в союз


## 3. Замена специальных символов на слова

In [13]:
def remove_alphabetic_symbols(col_name: str) -> str:
    restricted_symbols = [
        "(", 
        ")", 
        "+", 
        "-", 
        "[", 
        "]", 
        "/", 
        ".", 
        ",", 
        "@", 
        "{", 
        "}", 
        "«", 
        "»", 
        "<", 
        ">", 
        ":", 
        ";", 
        "'",
        "*",
        "?",
        "!",
        "-",
        "_"
    ]
    return "".join(c for c in col_name if not c.isalpha() and not c.isdigit() and not c.isspace() and c not in restricted_symbols)

non_alpha = df_cleansed["column_name"].progress_apply(lambda x: remove_alphabetic_symbols(str(x)))
non_alpha.unique()

100%|████████████████████████████| 4801706/4801706 [00:04<00:00, 1026750.08it/s]


array(['', '€', '−', '=', '№', '#', '±', '%', '—', '–', '$', '°', '\\',
       '…', '""', '\\№', '••', '∅', '|', '≈', '·', '्', '’', '%%', '\xad',
       '⊕', '&', '˦', '˩', '˧˩', '˥˧', '˥', '‰', '“”', '~%', 'ఁ', 'ం',
       'ః', '——', '~', '×', '£', '⋅', '&&', '=""', '¥', '#️⃣', '≡', '¬',
       '•', '№№——№—№——№——№№—№—№№—', '"', '⊙', '\xad\xad', '→↓', '""|&',
       '°—', '——————', '———————', '\xad\xad\xad', '’%', '%%%%%%', '•—',
       '‡', '––', '№№', '%—', '−−', '★', '·−', '½', '¼', '▼▲%', '†',
       '====', '××', '%%%%%%%%', '∞', '≥', '\x7f"``"\x7f', '……………………',
       '́', '½½', '→', '%–', '№\\', '⇅', '±%', '–%',
       '\xad\xad\xad\xad\xad', '\xad\xad\xad\xad', '°·', '꤫', '꤬', '꤭',
       '’—', '×−', '=̅', '♂', '♀', '∧', '———', '$$', '∆%', '：', '×⁻',
       '\u200e', '−·', '′′', '⠀', '§', 'ꤦ', 'ꤧ', 'ꤨ', 'ꤩ', 'ꤪ', '↓→',
       '≥°', '≤°', '́́', '₩', '=%', '—…', '↓·→', '%±', '▼', '||', '’’',
       '↕', '₽', '₨', '%~', '|||', '·°', '₣', 'ಂ', 'ಃ', '—°', '’’’', '×”',
       '￼', '

In [14]:
# # - тоже номер или не всегда
# Много знаков равенства, скорее всего в столбце число лежит, можно тоже указать
rules = {
    "№": " номер ",
    "%": " процент ",
    "∆": " число ",
    "∑": " сумма ",
    "Σ": " сумма ",
    "$": " деньги ",
    "₽": " деньги ",
    "€": " деньги ",
    "¥": " деньги ",
    "₩": " деньги ",
    "£": " деньги ",
}

for rule, word in rules.items():
    df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).replace(rule, word))

df_cleansed

100%|████████████████████████████| 4801706/4801706 [00:01<00:00, 2752549.37it/s]


,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,Чарт (2019)
1,dataset/Dataset/data/7779522/table_0_meta.json,1,Высшаяпозиция
3,dataset/Dataset/data/2863377/table_11_meta.json,1,Команда
4,dataset/Dataset/data/2863377/table_11_meta.json,2,И
5,dataset/Dataset/data/2863377/table_11_meta.json,3,В
...,...,...,...
409524,dataset/Dataset/data/35185/table_5_meta.json,2,Дата получения Любекского права
409525,dataset/Dataset/data/35185/table_5_meta.json,3,Дата получения Кульмского права
409526,dataset/Dataset/data/35185/table_5_meta.json,4,Дата получения Магдебургского права
409527,dataset/Dataset/data/35185/table_5_meta.json,5,Дата вступления в союз


## 3.1. To lower

In [15]:
df_cleansed["column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).lower())
df_cleansed

100%|████████████████████████████| 4801706/4801706 [00:02<00:00, 2323583.39it/s]


,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт (2019)
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция
3,dataset/Dataset/data/2863377/table_11_meta.json,1,команда
4,dataset/Dataset/data/2863377/table_11_meta.json,2,и
5,dataset/Dataset/data/2863377/table_11_meta.json,3,в
...,...,...,...
409524,dataset/Dataset/data/35185/table_5_meta.json,2,дата получения любекского права
409525,dataset/Dataset/data/35185/table_5_meta.json,3,дата получения кульмского права
409526,dataset/Dataset/data/35185/table_5_meta.json,4,дата получения магдебургского права
409527,dataset/Dataset/data/35185/table_5_meta.json,5,дата вступления в союз


## 4. Удаление символов не из русского алфавита

In [16]:
# for i in df_cleansed.sort_values(by=["column_name"], ascending=False)["column_name"]:
#     print(i)

In [17]:
def remove_non_alphabetic_symbols(col_name: str) -> str:
    # return "".join(c for c in col_name if c.isalpha() or c == " ")
    # return "".join(c for c in col_name if "А" <= c <= "я" or c == " ")
    return "".join(c for c in col_name if "а" <= c <= "я" or c == " ")

df_cleansed["column_name"] = df_cleansed["column_name"].progress_apply(lambda x: remove_non_alphabetic_symbols(str(x)))
df_cleansed

100%|█████████████████████████████| 4801706/4801706 [00:05<00:00, 893646.15it/s]


,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция
3,dataset/Dataset/data/2863377/table_11_meta.json,1,команда
4,dataset/Dataset/data/2863377/table_11_meta.json,2,и
5,dataset/Dataset/data/2863377/table_11_meta.json,3,в
...,...,...,...
409524,dataset/Dataset/data/35185/table_5_meta.json,2,дата получения любекского права
409525,dataset/Dataset/data/35185/table_5_meta.json,3,дата получения кульмского права
409526,dataset/Dataset/data/35185/table_5_meta.json,4,дата получения магдебургского права
409527,dataset/Dataset/data/35185/table_5_meta.json,5,дата вступления в союз


## 5. Удаление пробелов с начала и конца заголовка столбца

In [18]:
df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).strip())
df_cleansed = df_cleansed[df_cleansed["column_name"] != ""]

100%|████████████████████████████| 4801706/4801706 [00:01<00:00, 2973186.85it/s]


## 6. Удаление заголовков столбцов, используемых для группировки значений (по месяцу, году и т.п.)

In [59]:
group_by_columns = [
    "январь",
    "февраль",
    "март",
    "апрель",
    "май",
    "июнь",
    "июль",
    "август",
    "сентябрь",
    "октябрь",
    "ноябрь",
    "декабрь",
    "янв",
    "февр",
    "фев",
    "мар",
    "апр",
    "июн",
    "июл",
    "авг",
    "сент",
    "сен",
    "окт",
    "ноя",
    "нояб",
    "дек",
]

In [60]:
s = 0
for col in group_by_columns:
    s += df_cleansed[df_cleansed["column_name"] == col].shape[0]
s

21992

In [61]:
for col in group_by_columns:
    group_by_mask = df_cleansed["column_name"] == col
    df_cleansed = df_cleansed[~group_by_mask]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция
2,dataset/Dataset/data/2863377/table_11_meta.json,1,команда
3,dataset/Dataset/data/2863377/table_11_meta.json,9,очки
4,dataset/Dataset/data/4727026/table_0_meta.json,0,год
...,...,...,...
3393465,dataset/Dataset/data/35185/table_5_meta.json,2,дата получения любекского права
3393466,dataset/Dataset/data/35185/table_5_meta.json,3,дата получения кульмского права
3393467,dataset/Dataset/data/35185/table_5_meta.json,4,дата получения магдебургского права
3393468,dataset/Dataset/data/35185/table_5_meta.json,5,дата вступления в союз


In [62]:
(df_cleansed["column_name"] == "фев").sum()

0

## 7. Удаление заголовков длиной <= 2

In [63]:
len_le_2 = df_cleansed["column_name"].str.len() <= 2
df_cleansed[len_le_2]

,table_id,column_id,column_name


In [64]:
df_cleansed = df_cleansed[~len_le_2]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция
2,dataset/Dataset/data/2863377/table_11_meta.json,1,команда
3,dataset/Dataset/data/2863377/table_11_meta.json,9,очки
4,dataset/Dataset/data/4727026/table_0_meta.json,0,год
...,...,...,...
3393465,dataset/Dataset/data/35185/table_5_meta.json,2,дата получения любекского права
3393466,dataset/Dataset/data/35185/table_5_meta.json,3,дата получения кульмского права
3393467,dataset/Dataset/data/35185/table_5_meta.json,4,дата получения магдебургского права
3393468,dataset/Dataset/data/35185/table_5_meta.json,5,дата вступления в союз


In [65]:
df_cleansed = df_cleansed.reset_index().drop(["index"], axis=1)
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0_meta.json,0,чарт
1,dataset/Dataset/data/7779522/table_0_meta.json,1,высшаяпозиция
2,dataset/Dataset/data/2863377/table_11_meta.json,1,команда
3,dataset/Dataset/data/2863377/table_11_meta.json,9,очки
4,dataset/Dataset/data/4727026/table_0_meta.json,0,год
...,...,...,...
3371473,dataset/Dataset/data/35185/table_5_meta.json,2,дата получения любекского права
3371474,dataset/Dataset/data/35185/table_5_meta.json,3,дата получения кульмского права
3371475,dataset/Dataset/data/35185/table_5_meta.json,4,дата получения магдебургского права
3371476,dataset/Dataset/data/35185/table_5_meta.json,5,дата вступления в союз


In [95]:
from random import randint

rnd = randint(0, df_cleansed.shape[0])
df_cleansed.loc[rnd:rnd+10]

,table_id,column_id,column_name
2329947,dataset/Dataset/data/1723357/table_3_meta.json,0,сезоны
2329948,dataset/Dataset/data/1723357/table_3_meta.json,1,матчи
2329949,dataset/Dataset/data/1723357/table_3_meta.json,2,победы
2329950,dataset/Dataset/data/1723357/table_3_meta.json,3,ничьи
2329951,dataset/Dataset/data/1723357/table_3_meta.json,4,поражения
2329952,dataset/Dataset/data/1723357/table_3_meta.json,5,забитые голы
2329953,dataset/Dataset/data/1723357/table_3_meta.json,6,пропущенные голы
2329954,dataset/Dataset/data/1796802/table_1_meta.json,0,год
2329955,dataset/Dataset/data/1796802/table_1_meta.json,1,вышли во второй дивизион
2329956,dataset/Dataset/data/1796802/table_1_meta.json,2,вышли во второй дивизион


# Сохранение очищенного датафрейма

In [96]:
df_cleansed.to_csv("df_cleansed.csv", sep=";", index=False)